In [3]:
import pandas as pd
import psycopg2
import plotly.express as px
import requests
import json

In [4]:
# Dataframe Creation

db = psycopg2.connect(host = 'localhost',
                      user = 'postgres',
                      port = '5432',
                      database = 'PhonePe',
                      password = '1177')

cursor = db.cursor()


#aggregated_insurance_df

cursor.execute("SELECT * FROM aggregated_insurance")
db.commit()
table_agg_ins = cursor.fetchall()
aggregated_insurance = pd.DataFrame(table_agg_ins,columns=("States","Years","Quarter","Transaction_Type","Transaction_Count","Transaction_Amount"))

#aggregated_transaction_df

cursor.execute("SELECT * FROM aggregated_transaction")
db.commit()
table_agg_trans = cursor.fetchall()
aggregated_transaction = pd.DataFrame(table_agg_trans,columns=("States","Years","Quarter","Transaction_Type","Transaction_Count","Transaction_Amount"))

#aggregated_user_df

cursor.execute("SELECT * FROM aggregated_user")
db.commit()
table_agg_user = cursor.fetchall()
aggregated_user = pd.DataFrame(table_agg_user,columns=("States","Years","Quarter","brands","Transaction_Count","percentage"))

#map_insurance_df

cursor.execute("SELECT * FROM map_insurance")
db.commit()
table_map_ins = cursor.fetchall()
map_insurance = pd.DataFrame(table_map_ins,columns=("States","Years","Quarter","District","Transaction_Count","Transaction_Amount"))

#map_transaction_df

cursor.execute("SELECT * FROM map_transaction")
db.commit()
table_map_trans = cursor.fetchall()
map_transaction = pd.DataFrame(table_map_trans,columns=("States","Years","Quarter","District","Transaction_Count","Transaction_Amount"))

#map_user_df

cursor.execute("SELECT * FROM map_user")
db.commit()
table_map_user = cursor.fetchall()
map_user = pd.DataFrame(table_map_user,columns=("States","Years","Quarter","District","Registered_Users","App_Opens"))

#top_insurance_df

cursor.execute("SELECT * FROM top_insurance")
db.commit()
table_top_ins = cursor.fetchall()
top_insurance = pd.DataFrame(table_top_ins,columns=("States","Years","Quarter","Pincodes","Transaction_Count","Transaction_Amount"))

#top_transaction_df

cursor.execute("SELECT * FROM top_transaction")
db.commit()
table_top_trans = cursor.fetchall()
top_transaction = pd.DataFrame(table_top_trans,columns=("States","Years","Quarter","Pincodes","Transaction_Count","Transaction_Amount"))

#top_user_df

cursor.execute("SELECT * FROM top_user")
db.commit()
table_top_user = cursor.fetchall()
top_user = pd.DataFrame(table_top_user,columns=("States","Years","Quarter","Pincodes","Registered_Users"))



In [5]:
# transaction based on year
def Transaction_amount_count_year(df,year):
    tacy = df[df['Years'] == year]
    tacy.reset_index(drop=True,inplace=True)

    tacy_gr = tacy.groupby("States")[["Transaction_Count","Transaction_Amount"]].sum()
    tacy_gr.reset_index(inplace=True)

    amount_char = px.bar(tacy_gr,x="States",y="Transaction_Amount",title=f"Transaction Amount for {year}",
                        color_discrete_sequence=px.colors.sequential.Bluered_r,height=650,width=600)
    amount_char.show()

    count_char = px.bar(tacy_gr,x="States",y="Transaction_Count",title=f"Transaction Count for {year}",
                        color_discrete_sequence=px.colors.sequential.Bluered_r,height=650,width=600)
    count_char.show()

    map_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    res = requests.get(map_url)
    res_data = json.loads(res.content)
    state_name = []
    for feature in res_data['features']:
        state_name.append(feature['properties']['ST_NM'])
    state_name.sort()

    Ind_amount = px.choropleth(tacy_gr,geojson=res_data,locations="States",featureidkey="properties.ST_NM",
                        color="Transaction_Amount",color_continuous_scale="Rainbow",
                        range_color=(tacy_gr["Transaction_Amount"].min(),tacy_gr["Transaction_Amount"].max()),
                        hover_name="States",title=f"Transaction Amount for {year}",fitbounds="locations",height=650,width=600)
    Ind_amount.update_geos(visible = False)
    Ind_amount.show()

    Ind_count = px.choropleth(tacy_gr,geojson=res_data,locations="States",featureidkey="properties.ST_NM",
                        color="Transaction_Count",color_continuous_scale="Rainbow",
                        range_color=(tacy_gr["Transaction_Count"].min(),tacy_gr["Transaction_Count"].max()),
                        hover_name="States",title=f"Transaction Count for {year}",fitbounds="locations",height=650,width=600)
    Ind_count.update_geos(visible = False)
    Ind_count.show()

    return tacy

In [4]:
# transaction based on quarter
def Transaction_amount_count_quarter(df,quarter):
    tacq = df[df['Quarter'] == quarter]
    tacq.reset_index(drop=True,inplace=True)

    tacq_gr = tacq.groupby("States")[["Transaction_Count","Transaction_Amount"]].sum()
    tacq_gr.reset_index(inplace=True)

    amount_char = px.bar(tacq_gr,x="States",y="Transaction_Amount",title=f"Transaction Amount for year {tacq['Years'].min()} {quarter} Quarter",
                        color_discrete_sequence=px.colors.sequential.Bluered_r,height=650,width=600)
    amount_char.show()

    count_char = px.bar(tacq_gr,x="States",y="Transaction_Count",title=f"Transaction Count for year {tacq['Years'].min()} {quarter} Quarter",
                        color_discrete_sequence=px.colors.sequential.Bluered_r,height=650,width=600)
    count_char.show()

    map_url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    res = requests.get(map_url)
    res_data = json.loads(res.content)
    state_name = []
    for feature in res_data['features']:
        state_name.append(feature['properties']['ST_NM'])
    state_name.sort()

    Ind_amount = px.choropleth(tacq_gr,geojson=res_data,locations="States",featureidkey="properties.ST_NM",
                        color="Transaction_Amount",color_continuous_scale="Rainbow",
                        range_color=(tacq_gr["Transaction_Amount"].min(),tacq_gr["Transaction_Amount"].max()),
                        hover_name="States",title=f"Transaction Amount for year {tacq['Years'].min()} {quarter} Quarter",fitbounds="locations",height=650,width=600)
    Ind_amount.update_geos(visible = False)
    Ind_amount.show()

    Ind_count = px.choropleth(tacq_gr,geojson=res_data,locations="States",featureidkey="properties.ST_NM",
                        color="Transaction_Count",color_continuous_scale="Rainbow",
                        range_color=(tacq_gr["Transaction_Count"].min(),tacq_gr["Transaction_Count"].max()),
                        hover_name="States",title=f"Transaction Count for year {tacq['Years'].min()} {quarter} Quarter",fitbounds="locations",height=650,width=600)
    Ind_count.update_geos(visible = False)
    Ind_count.show()

    return tacq

In [5]:
# transaction based on transaction type
def aggregated_Transaction_type(df,state):

    tty = df[df['States'] == state]
    tty.reset_index(drop=True, inplace=True)

    tty_gr = tty.groupby("Transaction_Type")[["Transaction_Count","Transaction_Amount"]].sum()
    tty_gr.reset_index(inplace=True)

    trans_char_pie_amount = px.pie(tty_gr,"Transaction_Type","Transaction_Amount",width=600,title=f"Transaction Amount for {state}"
                            ,hole=0.5)
    trans_char_pie_amount.show()

    trans_char_pie_count = px.pie(tty_gr,"Transaction_Type","Transaction_Count",width=600,title=f"Transaction Count for {state}"
                            ,hole=0.5)
    trans_char_pie_count.show()

In [6]:
# aggregated user year
def aggregated_user_year(df,year):
    auy = df[df['Years']== year]
    auy.reset_index(drop=True,inplace=True)

    auy_gr = pd.DataFrame(auy.groupby("brands")["Transaction_Count"].sum())
    auy_gr.reset_index(inplace=True)

    auy_bar = px.bar(auy_gr,x="brands",y="Transaction_Count",title="Transaction Count of Brands",width=800,
                    color_discrete_sequence=px.colors.sequential.Bluered_r)
    auy_bar.show()

    return auy

In [7]:
# aggregated user quarter
def aggregated_user_year_quarter(df,quarter):
    auyq = df[df['Quarter']== quarter]
    auyq.reset_index(drop=True,inplace=True)

    auyq_gr = pd.DataFrame(auyq.groupby("brands")["Transaction_Count"].sum())
    auyq_gr.reset_index(inplace=True)

    auyq_bar = px.bar(auyq_gr,x="brands",y="Transaction_Count",title=f"Transaction Count of Brands {quarter} Quarter",width=800,
                        color_discrete_sequence=px.colors.sequential.Bluered_r)
    auyq_bar.show()
    
    return auyq

In [8]:
# aggregated user year quarter states
def aggregated_user_year_quarter_states(df,state):
    auyqs = df[df["States"] == state]
    auyqs.reset_index(drop=True,inplace=True)

    auyqs_char = px.pie(auyqs,"brands","Transaction_Count",width=600,title="Transaction Count of State",hover_data="percentage",hole=0.5)
    auyqs_char.show()

In [9]:
# map insurance district
def map_ins_dist(df,state):

    tty = df[df['States'] == state]
    tty.reset_index(drop=True, inplace=True)

    tty_gr = tty.groupby("District")[["Transaction_Count","Transaction_Amount"]].sum()
    tty_gr.reset_index(inplace=True)

    trans_char_bar_amount = px.bar(tty_gr,x="Transaction_Amount",y="District",orientation="h",title=f"District wise Transaction Amount Split for {state}",color_discrete_sequence=px.colors.sequential.Bluered_r)
    trans_char_bar_amount.show()

    trans_char_bar_count = px.bar(tty_gr,x="Transaction_Count",y="District",orientation="h",title=f"District wise Transaction Count Split for {state}",color_discrete_sequence=px.colors.sequential.Bluered_r)
    trans_char_bar_count.show()

In [10]:
# map user year
def map_user_year(df,year):
    muy = df[df['Years']== year]
    muy.reset_index(drop=True,inplace=True)

    muy_gr = pd.DataFrame(muy.groupby("States")[["Registered_Users","App_Opens"]].sum())
    muy_gr.reset_index(inplace=True)

    muy_reg_char = px.line(muy_gr,x="States",y=["Registered_Users","App_Opens"],title=f"Registered Users and App Opens Count for {year}",width=1000,height=800,markers=True)

    muy_reg_char.show()

    return muy

In [11]:
# map user year quarter
def map_user_year_quarter(df,quarter):
    muyq = df[df['Quarter']== quarter]
    muyq.reset_index(drop=True,inplace=True)

    muyq_gr = pd.DataFrame(muyq.groupby("States")[["Registered_Users","App_Opens"]].sum())
    muyq_gr.reset_index(inplace=True)

    muyq_reg_char = px.line(muyq_gr,x="States",y=["Registered_Users","App_Opens"],title=f"Registered Users and App Opens Chart for {df['Years'].min()} {quarter} Quarter",width=1000,height=800,markers=True)

    muyq_reg_char.show()

    return muyq

In [12]:
# map user year quarter states
def map_user_year_quarter_state(df,state):
    muyqs = df[df['States']== state]
    muyqs.reset_index(drop=True,inplace=True)

    muyqs_gr = pd.DataFrame(muyqs.groupby("District")[["Registered_Users","App_Opens"]].sum())
    muyqs_gr.reset_index(inplace=True)

    muyqs_reg_char = px.bar(muyqs_gr,x="Registered_Users",y="District",orientation="h",title=f"District wise Split for Registered User",color_discrete_sequence=px.colors.sequential.Bluered_r)
    muyqs_reg_char.show()

    muyqs_app_char = px.bar(muyqs_gr,x="App_Opens",y="District",orientation="h",title=f"District wise Split for App Opens",color_discrete_sequence=px.colors.sequential.Bluered_r)
    muyqs_app_char.show()


In [49]:
# top insurance pincode
def top_insu_tac_Y_P(df,state):
    tiy = df[df['States']== state]
    tiy.reset_index(drop=True,inplace=True)
    
    tiy_amount_char = px.bar(tiy,x="Quarter",y="Transaction_Amount",title=f"Pincode wise Transaction Amount Split",color_discrete_sequence=px.colors.sequential.thermal_r,hover_data="Pincodes")
    tiy_amount_char.show()

    tiy_count_char = px.bar(tiy,x="Quarter",y="Transaction_Count",title=f"Pincode wise Transaction Count Split",color_discrete_sequence=px.colors.sequential.thermal_r,hover_data="Pincodes")
    tiy_count_char.show()


In [66]:
# top user year 
def top_user_year(df,year):
    tuy = df[df['Years']== year]
    tuy.reset_index(drop=True,inplace=True)

    tuy_gr = tuy.groupby(["States","Quarter"])[["Registered_Users"]].sum()
    tuy_gr.reset_index(inplace=True)

    tuy_char = px.bar(tuy_gr,x="States",y="Registered_Users",color="Quarter",width=1000,height=800,
                    color_discrete_sequence=px.colors.sequential.thermal_r,hover_name="States",title=f"Registered Users Across States for {year}")

    tuy_char.show()

    return tuy

In [ ]:
# top use year and pincode
def top_user_year_pin(df,state):
    tuys = df[df['States']== state]
    tuys.reset_index(drop=True,inplace=True)

    tuys_char = px.bar(tuys,x="Quarter",y="Registered_Users",color="Registered_Users",width=1000,height=800,hover_data="Pincodes",
                    color_continuous_scale=px.colors.sequential.solar_r,title=f"Registered Users Pincodes Split for {state}")

    tuys_char.show()


### aggregated_insurance

In [ ]:
tac_Y = Transaction_amount_count_year(aggregated_insurance,2022)

In [ ]:
Transaction_amount_count_quarter(tac_Y,2)

### aggregated transaction

In [ ]:
aggre_trans_tac_Y = Transaction_amount_count_year(aggregated_transaction,2023)

In [ ]:
aggregated_Transaction_type(aggre_trans_tac_Y,"Puducherry")

In [ ]:
aggre_trans_tac_Y_Q = Transaction_amount_count_quarter(aggre_trans_tac_Y,3)

In [ ]:
aggregated_Transaction_type(aggre_trans_tac_Y_Q,"Puducherry")

### aggregated user

In [ ]:
agg_user_Y = aggregated_user_year(aggregated_user,2020)

In [ ]:
agg_user_Y_Q = aggregated_user_year_quarter(agg_user_Y,2)

In [ ]:
agg_user_Y_Q_S = aggregated_user_year_quarter_states(agg_user_Y_Q,"West Bengal")

### map insurance

In [ ]:
map_ins_tac_Y = Transaction_amount_count_year(map_insurance,2023)

In [ ]:
map_ins_dist(map_ins_tac_Y,"Andaman & Nicobar")

In [ ]:
map_ins_tac_Y_Q = Transaction_amount_count_quarter(map_ins_tac_Y,3)

In [ ]:
map_ins_dist(map_ins_tac_Y_Q,"West Bengal")

In [ ]:
map_insurance

### map transaction

In [ ]:
map_trans_tac_Y = Transaction_amount_count_year(map_transaction,2022)
map_trans_tac_Y

In [ ]:
map_ins_dist(map_trans_tac_Y,"Andaman & Nicobar")

In [ ]:
map_trans_tac_Y_Q = Transaction_amount_count_quarter(map_trans_tac_Y,3)

In [ ]:
map_ins_dist(map_trans_tac_Y_Q,"West Bengal")

### map user

In [ ]:
map_user_Y = map_user_year(map_user,2023)

In [ ]:
map_user_Y

In [ ]:
map_user_Y_Q = map_user_year_quarter(map_user_Y,2)
map_user_Y_Q

In [ ]:
map_user_year_quarter_state(map_user_Y_Q,"Andaman & Nicobar")

In [ ]:
map_user

### top insurance

In [ ]:
top_insu_tac_Y = Transaction_amount_count_year(top_insurance,2023)

In [ ]:
top_insu_tac_Y

In [ ]:
top_insu_tac_Y_P(top_insu_tac_Y,"West Bengal")

In [ ]:
top_insu_tac_Y_Q = Transaction_amount_count_quarter(top_insu_tac_Y,3)

In [ ]:
top_insurance

### top transaction

In [ ]:
top_trans_tac_Y = Transaction_amount_count_year(top_transaction,2023)

In [ ]:
top_insu_tac_Y_P(top_trans_tac_Y,"West Bengal")

In [ ]:
top_trans_tac_Y_Q = Transaction_amount_count_quarter(top_trans_tac_Y,3)

In [ ]:
top_transaction

### top user

In [ ]:
top_user_Y = top_user_year(top_user,2022)
top_user_Y

In [ ]:
top_user_year_pin(top_user_Y,"West Bengal")

### Top Charts Questions 

In [99]:
def top_chart_transaction_amount(table_name):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                        port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT states, SUM(transaction_amount) AS Transaction FROM {table_name}
            GROUP BY states
            ORDER BY Transaction DESC
            LIMIT 10;'''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("States","Transaction Amount"))
    df_q1_char = px.bar(df_q1,x="States",y="Transaction Amount",title=f"Top 10 States with Highest Transaction on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="States",height=650,width=600)
    df_q1_char.show()

    q2 = f'''SELECT states, SUM(transaction_amount) AS Transaction FROM {table_name}
            GROUP BY states
            ORDER BY Transaction
            LIMIT 10;'''

    cursor.execute(q2)
    table = cursor.fetchall()
    db.commit()

    df_q2 = pd.DataFrame(table,columns=("States","Transaction Amount"))
    df_q2_char = px.bar(df_q2,x="States",y="Transaction Amount",title=f"Top 10 States with Least Transaction on {table_name}",color_discrete_sequence=px.colors.sequential.thermal_r,
                    hover_name="States",height=650,width=600)
    df_q2_char.show()

    q3 = f'''SELECT states, AVG(transaction_amount) AS Transaction FROM {table_name}
            GROUP BY states
            ORDER BY Transaction;
            '''

    cursor.execute(q3)
    table = cursor.fetchall()
    db.commit()

    df_q3 = pd.DataFrame(table,columns=("States","Transaction Amount"))
    df_q3_char = px.bar(df_q3,x="Transaction Amount",y="States",title=f"Average Trend of Transaction on {table_name}",color_discrete_sequence=px.colors.sequential.Agsunset_r,
                    hover_name="States",height=650,width=600,orientation="h")
    df_q3_char.show()


In [107]:
def top_chart_transaction_count(table_name):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                        port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT states, SUM(transaction_count) AS Transaction FROM {table_name}
            GROUP BY states
            ORDER BY Transaction DESC
            LIMIT 10;'''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("States","Transaction Count"))
    df_q1_char = px.bar(df_q1,x="States",y="Transaction Count",title=f"Top 10 States with Highest Transaction Count on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="States",height=800,width=1000)
    df_q1_char.show()

    q2 = f'''SELECT states, SUM(transaction_count) AS Transaction FROM {table_name}
            GROUP BY states
            ORDER BY Transaction
            LIMIT 10;'''

    cursor.execute(q2)
    table = cursor.fetchall()
    db.commit()

    df_q2 = pd.DataFrame(table,columns=("States","Transaction Count"))
    df_q2_char = px.bar(df_q2,x="States",y="Transaction Count",title=f"Top 10 States with Least Transaction Count on {table_name}",color_discrete_sequence=px.colors.sequential.thermal_r,
                    hover_name="States",height=800,width=1000)
    df_q2_char.show()

    q3 = f'''SELECT states, AVG(transaction_count) AS Transaction FROM {table_name}
            GROUP BY states
            ORDER BY Transaction;
            '''

    cursor.execute(q3)
    table = cursor.fetchall()
    db.commit()

    df_q3 = pd.DataFrame(table,columns=("States","Transaction Count"))
    df_q3_char = px.bar(df_q3,x="Transaction Count",y="States",title=f"Average Trend of Transaction Count on {table_name}",color_discrete_sequence=px.colors.sequential.Agsunset_r,
                    hover_name="States",height=800,width=1000,orientation="h")
    df_q3_char.show()


In [ ]:
top_chart_transaction_count("map_insurance")

In [15]:
def top_chart_reg_user(table_name,state):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                        port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT district,SUM(registered_user) AS Registered_Users FROM {table_name}
                WHERE states = '{state}'
                GROUP BY district
                ORDER BY Registered_Users DESC
                LIMIT 10
                ;'''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("District","Registered Users"))
    df_q1_char = px.bar(df_q1,x="District",y="Registered Users",title=f"Top 10 States with Highest Registered Users on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="District",height=650,width=600)
    df_q1_char.show()

    q2 = f'''SELECT district,SUM(registered_user) AS Registered_Users FROM {table_name}
                WHERE states = '{state}'
                GROUP BY district
                ORDER BY Registered_Users
                LIMIT 10;'''

    cursor.execute(q2)
    table = cursor.fetchall()
    db.commit()

    df_q2 = pd.DataFrame(table,columns=("District","Registered Users"))
    df_q2_char = px.bar(df_q2,x="District",y="Registered Users",title=f"Top 10 States with Least Registered Users on {table_name}",color_discrete_sequence=px.colors.sequential.thermal_r,
                    hover_name="District",height=650,width=600)
    df_q2_char.show()

    q3 = f'''SELECT district,SUM(registered_user) AS Registered_Users FROM {table_name}
                WHERE states = '{state}'
                GROUP BY district
                ORDER BY Registered_Users
                ;'''

    cursor.execute(q3)
    table = cursor.fetchall()
    db.commit()

    df_q3 = pd.DataFrame(table,columns=("District","Registered Users"))
    df_q3_char = px.bar(df_q3,x="Registered Users",y="District",title=f"Average Trend of Registered Users on {table_name}",color_discrete_sequence=px.colors.sequential.Agsunset_r,
                    hover_name="District",height=650,width=600,orientation="h")
    df_q3_char.show()


In [ ]:
top_chart_reg_user("map_user",'West Bengal')

In [19]:
def top_chart_app_opens_user(table_name,state):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                        port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT district,SUM(app_opens) AS App_Opens FROM {table_name}
                WHERE states = '{state}'
                GROUP BY district
                ORDER BY app_opens DESC
                LIMIT 10
                ;'''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("District","App Opens"))
    df_q1_char = px.bar(df_q1,x="District",y="App Opens",title=f"Top 10 States with Highest App Opens on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="District",height=650,width=600)
    df_q1_char.show()

    q2 = f'''SELECT district,SUM(app_opens) AS App_Opens FROM {table_name}
                WHERE states = '{state}'
                GROUP BY district
                ORDER BY app_opens
                LIMIT 10;'''

    cursor.execute(q2)
    table = cursor.fetchall()
    db.commit()

    df_q2 = pd.DataFrame(table,columns=("District","App Opens"))
    df_q2_char = px.bar(df_q2,x="District",y="App Opens",title=f"Top 10 States with Least App Opens on {table_name}",color_discrete_sequence=px.colors.sequential.thermal_r,
                    hover_name="District",height=650,width=600)
    df_q2_char.show()

    q3 = f'''SELECT district,SUM(app_opens) AS App_Opens FROM {table_name}
                WHERE states = '{state}'
                GROUP BY district
                ORDER BY app_opens
                ;'''

    cursor.execute(q3)
    table = cursor.fetchall()
    db.commit()

    df_q3 = pd.DataFrame(table,columns=("District","App Opens"))
    df_q3_char = px.bar(df_q3,x="App Opens",y="District",title=f"Average Trend of App Opens on {table_name}",color_discrete_sequence=px.colors.sequential.Agsunset_r,
                    hover_name="District",height=650,width=600,orientation="h")
    df_q3_char.show()


In [ ]:
top_chart_app_opens_user('map_user','Tamil Nadu')

In [22]:
def top_chart_reg_user_map(table_name):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                        port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT states,SUM(registered_users) AS registered_users FROM {table_name}
                GROUP BY states
                ORDER BY registered_users DESC
                LIMIT 10;'''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("States","Registered Users"))
    df_q1_char = px.bar(df_q1,x="States",y="Registered Users",title=f"Top 10 States with Highest Registered Users on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="States",height=650,width=600)
    df_q1_char.show()

    q2 = f'''SELECT states,SUM(registered_users) AS registered_users FROM {table_name}
                GROUP BY states
                ORDER BY registered_users
                LIMIT 10;'''

    cursor.execute(q2)
    table = cursor.fetchall()
    db.commit()

    df_q2 = pd.DataFrame(table,columns=("States","Registered Users"))
    df_q2_char = px.bar(df_q2,x="States",y="Registered Users",title=f"Top 10 States with Least Registered Users on {table_name}",color_discrete_sequence=px.colors.sequential.thermal_r,
                    hover_name="States",height=650,width=600)
    df_q2_char.show()

    q3 = f'''SELECT states,AVG(registered_users) AS registered_users FROM {table_name}
                GROUP BY states
                ORDER BY registered_users
                ;'''

    cursor.execute(q3)
    table = cursor.fetchall()
    db.commit()

    df_q3 = pd.DataFrame(table,columns=("States","Registered Users"))
    df_q3_char = px.bar(df_q3,x="Registered Users",y="States",title=f"Average Trend of Registered Users on {table_name}",color_discrete_sequence=px.colors.sequential.Agsunset_r,
                    hover_name="States",height=650,width=600,orientation="h")
    df_q3_char.show()


In [ ]:
top_chart_reg_user_map("top_user")

In [8]:
def top_chart_transaction_type(table_name,transaction_type):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                         port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT states,SUM(transaction_amount) AS total_amount FROM {table_name}
                WHERE transaction_type = '{transaction_type}'
                GROUP BY states
                ORDER BY total_amount DESC
                LIMIT 5;
                '''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("States","Transaction Amount"))
    df_q1_char = px.bar(df_q1,x="States",y="Transaction Amount",title=f"Top 10 States with Highest Transaction on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="States",height=650,width=600)
    df_q1_char.show()

In [ ]:
top_chart_transaction_type("aggregated_transaction",'Peer-to-peer payments')

In [6]:
def top_chart_district(table_name):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                         port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT district,states,SUM(transaction_amount) AS total_transaction
                FROM {table_name}
                GROUP BY district,states
                ORDER BY total_transaction DESC
                LIMIT 10;
                '''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("District","States","Transaction Amount"))
    df_q1_char = px.bar(df_q1,x="District",y="Transaction Amount",title=f"Top 10 Districts with Highest Transaction on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="States",height=800,width=800)
    df_q1_char.show()

    q2 = f'''SELECT district,states,SUM(transaction_amount) AS total_transaction
                FROM {table_name}
                GROUP BY district,states
                ORDER BY total_transaction
                LIMIT 10;
                '''

    cursor.execute(q2)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("District","States","Transaction Amount"))
    df_q1_char = px.bar(df_q1,x="District",y="Transaction Amount",title=f"Top 10 Districts with Least Transaction on {table_name}",color_discrete_sequence=px.colors.sequential.Bluered_r,
                    hover_name="States",height=800,width=800)
    df_q1_char.show()

In [ ]:
top_chart_district("map_transaction")

In [26]:
def top_chart_brands(table_name):
    db = psycopg2.connect(host = 'localhost',
                        user = 'postgres',
                         port = '5432',
                        database = 'PhonePe',
                        password = '1177')

    cursor = db.cursor()

    q1 = f'''SELECT
                states,
                brands,
                total_transaction
            FROM (
                SELECT
                    states,
                    brands,
                    SUM(transaction_count) AS total_transaction,
                    ROW_NUMBER() OVER (PARTITION BY states ORDER BY SUM(transaction_count) DESC) AS rank
                FROM
                    {table_name}
                GROUP BY
                    states, brands
            ) ranked_data
            WHERE rank = 1
            ORDER BY total_transaction DESC;

                            '''

    cursor.execute(q1)
    table = cursor.fetchall()
    db.commit()

    df_q1 = pd.DataFrame(table,columns=("States","brands","Total Transaction"))
    df_q1_char = px.bar(df_q1,x="States",y="Total Transaction",title=f"Top 10 States and the respective Phone Brands used to make Transaction",color_discrete_sequence=px.colors.sequential.Rainbow_r,
                    hover_name="brands",height=800,width=800,color="brands")
    df_q1_char.show()
    

In [ ]:
top_chart_district("aggregated_user")

In [31]:
map_transaction

,States,Years,Quarter,District,Transaction_Count,Transaction_Amount
0,Andaman & Nicobar,2018,1,north and middle andaman district,442,931663
1,Andaman & Nicobar,2018,1,south andaman district,5688,12560249
2,Andaman & Nicobar,2018,1,nicobars district,528,1139849
3,Andaman & Nicobar,2018,2,north and middle andaman district,825,1317863
4,Andaman & Nicobar,2018,2,south andaman district,9395,23948236
...,...,...,...,...,...,...
19023,West Bengal,2024,2,nadia district,49543213,85889425747
19024,West Bengal,2024,2,birbhum district,29485233,49694427747
19025,West Bengal,2024,2,purba medinipur district,55342585,98897892001
19026,West Bengal,2024,2,maldah district,52271735,90432057728
